In [1]:
import torch
import triton
import triton.language as tl

In [ ]:
@triton.jit
def _layer_norm_fwd_fused(
    X,Y,W,B,Mean,Rstd,
    stride,N,eps,BLOCK_SIZE:tl.constexpr,):
    row = tl.program_id(0)
    Y += row * stride
    X += row * stride

    # Compute mean
    mean = 0
    _mean = tl.zeros([BLOCK_SIZE],dtype=tl.float32)
    for off in range(0,N,BLOCK_SIZE):
        cols = off + tl.arange(0,BLOCK_SIZE)
        a = tl.load(X + cols,mask=cols < N,other=0.0).to(tl.float32)
        _mean += a
    mean = tl.sum(_mean,axis=0) / N

    # Compute variance
    _var = tl.zeros([BLOCK_SIZE],dtype=tl.float32)
    for off in range(0,N, 
    
    